In [173]:
import pandas as pd
import numpy as np
import os

In [ ]:
def get_dictionaries(filename:str) -> dict:
    """
    Reads a CSV file and returns a dictionary with the first column as keys and the second column as values.
    
    Args:
        filename (str): The path to the CSV file.
        
    Returns:
        dict: A dictionary with the first column as keys and the second column as values.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(filename, header=None)
    
    # Create a dictionary from the DataFrame
    dictionary = dict(zip(df.iloc[:, 0], df.iloc[:, 1]))
    
    return dictionary

dictionaries = {}
path = "data/ExtraccionDomiVersionFinal/"
for file in os.listdir(path):
    # Get the full path of the file
    file_path = os.path.join(path, file)
    
    # Check if the file exists
    if os.path.exists(file_path) and file.startswith("Cat"):
        # Read the dictionary from the file
        dictionaries[file] = get_dictionaries(file_path)
    else:
        print(f"File {file} does not exist.")

File ListaCobro.csv does not exist.
File ListaCobroDetalle2023.csv does not exist.
File ListaCobroDetalle2022.csv does not exist.
File ListaCobroDetalle2025.csv does not exist.
File ListaCobroDetalle2024.csv does not exist.


In [175]:
dictionaries.keys()

dict_keys(['ListaCobroEmisora.csv', 'CatRespuestaBancos.csv', 'CatBanco.csv', 'CatEmisora.csv'])

In [176]:
datos_extraccion = {}
path = 'data/ExtraccionDomiVersionFinal/'
for archivo in os.listdir(path):
    if archivo.endswith('.csv') and not archivo.startswith("Cat"):
        print(f"Processing file: {archivo}")
        datos_extraccion[archivo] = pd.read_csv(os.path.join(path, archivo))

Processing file: ListaCobro.csv
Processing file: ListaCobroEmisora.csv
Processing file: ListaCobroDetalle2023.csv


/var/folders/tl/n2n7gv595bq_stcnjf2_yz440000gn/T/ipykernel_66151/1391424003.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  datos_extraccion[archivo] = pd.read_csv(os.path.join(path, archivo))


Processing file: ListaCobroDetalle2022.csv
Processing file: ListaCobroDetalle2025.csv
Processing file: ListaCobroDetalle2024.csv


In [177]:
datos_extraccion = dict(sorted(datos_extraccion.items()))

In [178]:
fechas = datos_extraccion['ListaCobro.csv']
lista_detalles = datos_extraccion["ListaCobroDetalle2022.csv"]
emisora = datos_extraccion["ListaCobroEmisora.csv"]

In [179]:
for file in datos_extraccion:
    #If file contains number in name concat with lista_detalles
    if file != "ListaCobroDetalle2022.csv" and any(char.isdigit() for char in file):
        print(file)
        lista_detalles = pd.concat([lista_detalles, datos_extraccion[file]])

ListaCobroDetalle2023.csv
ListaCobroDetalle2024.csv
ListaCobroDetalle2025.csv


In [207]:
cobros_final = lista_detalles.merge(
	fechas,
	on='idListaCobro',
	how='left',
	suffixes=('', '_fechas')
).drop(columns = "idBanco_fechas").merge(emisora, on='idListaCobro', how='left')


In [208]:
cobros_final["idBanco"] = cobros_final["idBanco"].astype(str).map(dictionaries["CatBanco.csv"]).fillna(cobros_final["idBanco"])
cobros_final["idRespuestaBanco"] = cobros_final["idRespuestaBanco"].astype(str).map(dictionaries["CatRespuestaBancos.csv"]).fillna(cobros_final["idRespuestaBanco"])
cobros_final["idEmisora"] = cobros_final["idEmisora"].astype(str).map(dictionaries["CatEmisora.csv"]).fillna(cobros_final["idEmisora"])